![Digital Futures](https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true)

# Setting up the Project Environment

Ultimately, the Python code written here will be extracted to scripts for execution in an automated pipeline.  To facilitate this, there is a need to set up a project environment that will allow for the execution of the code in a controlled and reproducible environment.

In the initial stages of the activities, the packages needed are `requests` and `pytest`.  The `requests` package is used to make HTTP requests to the API, while `pytest` is used for testing the code we also need *BeautifulSoup* (package name `beautifulsoup4`.  In later activities, you may need to install additional packages.  To do this, add the packages to the `pip install` command below and re-run the cell.

> **Remember:** The goal is to create a set of code cells that can be extracted to separate scripts for execution in an automated pipeline.  Therefore, the code should be kept in 3 distinct cells:
> 
> - **Shell Commands**:  Used to set up the project environment
> 
> - **Python Tests**: Used to test the Python production scripts both now and as part of the automated pipeline
> 
> - **Python Production Code**: The Python code that will be extracted to a script to execute during the pipeline

---

# Environment Setup Scripts

If you are running this notebook after cloning and have not set up your environment to run shell commands, you will need to run the following commands in your terminal to set up the environment.

> **NOTE:**  These commands need to be executed in the terminal.  
>
> Open a terminal at the root of your project before executing these commands
> 
> Until your environment is set up, Jupyter Notebooks will not be able to run **shell** scripts.

```sh
# Create a virtual environment (add the command below)
python3 -m venv .venv # Note: This command could also be python -m venv .venv # python3 and python are a symlink to the python version installed on your system

# Activate the virtual environment 
source .venv/bin/activate

# Install required package to execute shell commands from Jupyter Notebook
pip install ipykernel               ## OR 
pip install -r requirements.txt     ## IF there is already a requirements.txt file CONTAINING ipykenrnel in the project
```


In [6]:
# Install the necessary packages
!pip install requests pytest beautifulsoup4

# Create a requirements.txt file
!pip freeze > requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached charset_normalizer-3.4.0-cp313-cp313-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached charset_normalizer-3.4.0-cp313-cp313-win_amd64.whl (102 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)


> **Note:** 
> The `!` at the beginning of the lines is a special character in Jupyter Notebooks that allows you to run shell commands from the notebook.  
> These will need to be removed from any commands that are to be exported to a `.sh` shell script file for the pipeline.

---

# Python Tests

Develop any tests for functions in separate cells below.  The first has been provided for you as an example, add others as necessary.

### Test `request_to_scrape`

In [24]:
# Test request_to_scrape
import pytest
from unittest.mock import patch
from requests.exceptions import Timeout, RequestException

def test_request_to_scrape_makes_correct_request():
  # Arrange
  url = 'http://books.toscrape.com/'

  with patch('requests.get') as mock_get:
        mock_get.return_value.status_code = 200
        # Act
        request_to_scrape(url)

        # Assert
        mock_get.assert_called_with(url)


### Run the tests

Run the cell containing the `ipytest.run()` command to execute the tests.  The tests should all fail until you have written the production code.

Don't forget to run the installation and initialisation cell too on the first time you run the tests!


---

# Python Production Code


Develop any functions for use as production code in separate cells below. The first has been provided as an example under the Production Constants, add others as necessary.

### PRODUCTION CONSTANTS

In [8]:
# PRODUCTION CONSTANTS

# Constants for status messages
STATUS_SUCCESS = "success"
STATUS_ERROR = "error"
ERROR_NOT_HTML = "The response is not HTML"
ERROR_REQUEST_FAILED = "Request failed for URL"
ERROR_UNEXPECTED = "Unexpected error for URL"

# HTML Parser
HTML_PARSER = "html.parser"

### `request_to_scrape` Production Code

In [20]:
# request_to_scrape Production Code
def request_to_scrape(url):
  return None


---

# Python Execution Code

Develop any code to call the developed functions below.  Add additional cells so you don't need to re-run all of the code when you develop further scripts.

In [10]:
# Python Execution Code


---

# Jupyter Notebook Test and Linting Set Up

To run `pytest` scripts in a Jupyter Notebook cell, we need to install the `ipytest` package.  This package is NOT required for a pipeline and therefore it can be removed from the `requirements.txt` file before adding the production code to the pipeline.

To run linting, we need to install 2 packages `nbqa` and `flake8`.  We will make sure that `flake8` is included in the `requirements.txt` file when constructing the pipeline so that we can lint as part of the pipeline tests.

Run the following cell to install the `ipytest`, `nbqa` and `flake8` packages and a coverage package to help determine if all of your production code is executed during the tests!

This cell only needs to be run once (or after restarting the notebook kernel) to set up the environment for testing and linting.


In [11]:
# Install the `ipytest`, `nbqa` and `flake8` packages
!pip install ipytest nbqa flake8

  Using cached flake8-7.1.1-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached mccabe-0.7.0-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached pyflakes-3.2.0-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached flake8-7.1.1-py2.py3-none-any.whl (57 kB)
Using cached mccabe-0.7.0-py2.py3-none-any.whl (7.3 kB)
Using cached pyflakes-3.2.0-py2.py3-none-any.whl (62 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Set up `ipytest` to execute `pytest` scripts in Jupyter Notebook

In [12]:
# Configure ipytest for Jupyter Notebook

import ipytest
ipytest.autoconfig(rewrite_asserts=True, magics=True)

### Create a *config* file for `flake8`

Run this script to create a file in your project root

In [13]:
# Create a config file and ignore some flake8 rules
!echo "[flake8]" > .flake8
!echo "ignore = E402, W291, F811" >> .flake8

# Execute the tests and linting in the Jupyter Notebook

Run the following cell ***EVERY TIME*** you want to run the tests and linting that you have written in the *Python Tests* cell above.

>**Note:**
>
> This entire section does not need to be part of any pipeline scripts.  
> It is only required for the Jupyter Notebook environment during development.


## Run the tests

In [25]:
# Run the tests
ipytest.run("-vv", "-ss")

======================================= test session starts =======================================
platform win32 -- Python 3.13.0, pytest-8.3.3, pluggy-1.5.0 -- c:\Users\afzal\Desktop\Digital Futures\Python\web-scraping-sen-mo\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\afzal\Desktop\Digital Futures\Python\web-scraping-sen-mo
collecting ... collected 1 item

t_aace512d50c64b84ba6ec5fc4778fc78.py::test_request_to_scrape_makes_correct_request FAILED

============================================ FAILURES =============================================
__________________________ test_request_to_scrape_makes_correct_request ___________________________

    def test_request_to_scrape_makes_correct_request():
      # Arrange
      url = 'http://books.toscrape.com/'
    
      with patch('requests.get') as mock_get:
            mock_get.return_value.status_code = 200
            # Act
            request_to_scrape(url)
    
            # Assert
>           mock_get.asser

<ExitCode.TESTS_FAILED: 1>

## Run the linter

Run this script each time you want to lint your code

In [15]:
# Run the linter
!nbqa flake8 --show-source --format=pylint webscraping.ipynb

webscraping.ipynb:cell_2:3: [E114] indentation is not a multiple of 4 (comment)
  # Arrange
  ^
webscraping.ipynb:cell_2:4: [E111] indentation is not a multiple of 4
  url = 'http://books.toscrape.com/'
  ^
webscraping.ipynb:cell_2:6: [E111] indentation is not a multiple of 4
  with patch('requests.get') as mock_get:
  ^
webscraping.ipynb:cell_2:6: [F821] undefined name 'patch'
  with patch('requests.get') as mock_get:
       ^
webscraping.ipynb:cell_2:7: [E111] indentation is not a multiple of 4
          mock_get.return_value.status_code = 200
          ^
webscraping.ipynb:cell_2:7: [E117] over-indented
          mock_get.return_value.status_code = 200
          ^
webscraping.ipynb:cell_2:8: [E114] indentation is not a multiple of 4 (comment)
  # Act
  ^
webscraping.ipynb:cell_2:9: [F821] undefined name 'request_to_scrape'
  request_to_scrape(url)
  ^
webscraping.ipynb:cell_2:9: [E111] indentation is not a multiple of 4
  request_to_scrape(url)
  ^
webscraping.ipynb:cell_2:10: [E114]


---
